In [4]:
!cd C:\MT && git init && git branch -M main
# создайте .gitignore вручную или так:
_ = open(r"C:\MT\.gitignore","w",encoding="utf-8").write("""<вставьте содержимое из ответа>""")
!cd C:\MT && git add . && git commit -m "Initial commit"
!cd C:\MT && git remote add origin https://github.com/<user>/<repo>.git
!cd C:\MT && git push -u origin main

�� ����� "C:\Program Files\Git\cmd\git.exe" �� ᮢ���⨬� � ���ᨥ� Windows, ࠡ���饩 �� �⮬ ��������. �஢���� ᢥ����� � ��⥬�, � ��⥬ ������� � ����⥫� �ணࠬ����� ���ᯥ祭��.
�� ����� "C:\Program Files\Git\cmd\git.exe" �� ᮢ���⨬� � ���ᨥ� Windows, ࠡ���饩 �� �⮬ ��������. �஢���� ᢥ����� � ��⥬�, � ��⥬ ������� � ����⥫� �ணࠬ����� ���ᯥ祭��.
�� 㤠���� ���� 㪠����� 䠩�.
�� ����� "C:\Program Files\Git\cmd\git.exe" �� ᮢ���⨬� � ���ᨥ� Windows, ࠡ���饩 �� �⮬ ��������. �஢���� ᢥ����� � ��⥬�, � ��⥬ ������� � ����⥫� �ணࠬ����� ���ᯥ祭��.


In [4]:
# Выполните в терминале или в ячейке Jupyter с !
!pip install --upgrade pip
!pip install  fastapi uvicorn python-telegram-bot pydub scikit-learn pandas numpy matplotlib seaborn tqdm soundfile aiofiles python-multipart sqlite-utils

  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Users\MIKHAIL\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Обновите необходимые библиотеки
!pip install --upgrade librosa audioread soundfile


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from pydub import AudioSegment
from pydub.utils import which

AudioSegment.converter = which("ffmpeg") or r"C:\ffmpeg\ffmpeg-2025-09-10-git-c1dc2e2b7c-full_build\bin\ffmpeg.exe"
print("FFmpeg подключен:", AudioSegment.converter)


FFmpeg подключен: C:\ffmpeg\ffmpeg-2025-09-10-git-c1dc2e2b7c-full_build\bin\ffmpeg.exe


In [8]:
# Проверяем, что ключевые библиотеки импортируются
packages = ["torch", "torchaudio", "librosa", "fastapi", "uvicorn", "telegram", "pydub", "sklearn", "pandas", "numpy"]

import importlib
missing = []

for p in packages:
    try:
        importlib.import_module(p)
    except Exception as e:
        missing.append((p, str(e)))

print('Missing or failed imports:', missing)


Missing or failed imports: []


In [9]:
import torch

print(f"Версия PyTorch: {torch.__version__}")
print(f"CUDA доступно: {torch.cuda.is_available()}")
print(f"Версия CUDA: {torch.version.cuda if torch.cuda.is_available() else 'N/A'}")
print(f"Количество GPU: {torch.cuda.device_count()}")
print(f"Имя GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")

# Дополнительная проверка
if torch.cuda.is_available():
    print("\nДополнительная информация о GPU:")
    print(f"Объем памяти: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"Вычислительная способность: {torch.cuda.get_device_properties(0).major}.{torch.cuda.get_device_properties(0).minor}")
else:
    print("\n❌ CUDA недоступно. Возможные причины:")
    print("1. Несовместимость версий Python/CUDA/PyTorch")
    print("2. Отсутствие драйверов NVIDIA")
    print("3. Проблемы с установкой CUDA Toolkit")

Версия PyTorch: 2.8.0+cu128
CUDA доступно: True
Версия CUDA: 12.8
Количество GPU: 1
Имя GPU: NVIDIA GeForce GTX 1650

Дополнительная информация о GPU:
Объем памяти: 4.00 GB
Вычислительная способность: 7.5


In [10]:
# Основные импорты
import os
import io
import json
import time
import math
import logging
from pathlib import Path


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchaudio


import librosa
import soundfile as sf


from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split


# Для backend и бота
from fastapi import FastAPI, UploadFile, File, HTTPException, Request
from fastapi.responses import JSONResponse
import uvicorn
from pydantic import BaseModel


from telegram import Update, ReplyKeyboardMarkup, KeyboardButton
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes


# Асинхронные файлы
import aiofiles


# Логи
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("app.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger('music_genre')


# Константы
SUPPORTED_FORMATS = {'.mp3', '.wav', '.ogg'}
MAX_FILE_SIZE = 20 * 1024 * 1024 # 20 MB
# Детальная проверка GPU
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print(f'GPU доступен: {torch.cuda.get_device_name(0)}')
    print(f'Количество GPU: {torch.cuda.device_count()}')
    print(f'Текущий GPU: {torch.cuda.current_device()}')
else:
    DEVICE = torch.device('cpu')
    print('GPU не доступен, используется CPU')

print('Device:', DEVICE)
# Функции для мониторинга и оптимизации памяти GPU
def print_gpu_memory():
    """Выводит информацию об использовании памяти GPU"""
    if torch.cuda.is_available():
        print(f"Использовано памяти GPU: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
        print(f"Зарезервировано памяти GPU: {torch.cuda.memory_reserved()/1024**3:.2f} GB")

def clear_gpu_memory():
    """Очищает кэш памяти GPU и системную память"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    import gc
    gc.collect()
    print("Память очищена")

GPU доступен: NVIDIA GeForce GTX 1650
Количество GPU: 1
Текущий GPU: 0
Device: cuda


In [8]:
Система принимает аудиофайл (mp3/wav/ogg, ≤20MB) → преобразует в спектрограмму (MFCC / Mel) → модель (CNN + RNN) классифицирует жанр (10 классов) → результат возвращается через FastAPI и Telegram-бота. Также реализована возможность обратной связи.


Требования по точности и времени инференса: в идеале accuracy ≥ 90% (в реальности зависит от данных и объёма обучения), время инференса ≤ 2 с на CPU (на GPU быстрее).

SyntaxError: invalid character '≤' (U+2264) (2300786228.py, line 1)

In [17]:
from pathlib import Path

# Вариант 1 — двойные слэши
DATA_DIR = Path("C:\\archive\\Data\\genres_original")

# Вариант 2 — сырая строка (лучший вариант на Windows)
# DATA_DIR = Path(r"C:\archive\Data\genres_original")

if not DATA_DIR.exists():
    print("Папка не найдена:", DATA_DIR)
else:
    print("Датасет найден:", DATA_DIR)


Датасет найден: C:\archive\Data\genres_original


In [12]:
# Сканируем папку и сохраняем записи

def list_audio_files(root: Path, exts=SUPPORTED_FORMATS, max_size=MAX_FILE_SIZE):
    rows = []
    for genre_dir in root.iterdir():
        if not genre_dir.is_dir():
            continue
        genre = genre_dir.name
        for f in genre_dir.glob('*'):
            if f.suffix.lower() in exts:
                size = f.stat().st_size
                if size <= max_size:
                    rows.append({'path': str(f), 'genre': genre, 'size': size})
    return pd.DataFrame(rows)

if DATA_DIR.exists():
    df_files = list_audio_files(DATA_DIR)
    print('Found', len(df_files), 'audio files')
    display(df_files.head())

Found 1000 audio files


,path,genre,size
0,C:\archive\Data\genres_original\blues\blues.00...,blues,1323632
1,C:\archive\Data\genres_original\blues\blues.00...,blues,1323632
2,C:\archive\Data\genres_original\blues\blues.00...,blues,1323632
3,C:\archive\Data\genres_original\blues\blues.00...,blues,1323632
4,C:\archive\Data\genres_original\blues\blues.00...,blues,1323632


In [13]:
# Проверка баланса
if 'df_files' in globals():
    display(df_files['genre'].value_counts())

# Увеличим количество примеров на класс
TARGET_PER_CLASS = 200  # Увеличиваем с 100 до 200

def balance_dataframe(df, per_class=TARGET_PER_CLASS):
    grouped = df.groupby('genre')
    out = []
    for genre, g in grouped:
        if len(g) >= per_class:
            out.append(g.sample(per_class, random_state=42))
        else:
            # upsample with replacement
            out.append(g.sample(per_class, replace=True, random_state=42))
    return pd.concat(out).reset_index(drop=True)

if 'df_files' in globals():
    df_bal = balance_dataframe(df_files, per_class=TARGET_PER_CLASS)
    print('After balancing:', df_bal['genre'].value_counts())


genre
blues        100
classical    100
country      100
disco        100
hiphop       100
jazz         100
metal        100
pop          100
reggae       100
rock         100
Name: count, dtype: int64

After balancing: genre
blues        200
classical    200
country      200
disco        200
hiphop       200
jazz         200
metal        200
pop          200
reggae       200
rock         200
Name: count, dtype: int64


In [14]:
# Проверка файлов перед созданием датасета
valid_files = []
for idx, row in df_bal.iterrows():
    try:
        y = load_audio(row['path'])
        if len(y) > 0:  # Проверяем, что файл не пустой
            valid_files.append(idx)
    except:
        print(f"Пропускаем файл: {row['path']}")

df_bal = df_bal.loc[valid_files].reset_index(drop=True)

Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00051.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00092.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00014.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00071.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00060.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00020.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00082.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00086.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00074.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00074.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00087.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00099.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00023.wav
Пропускаем файл: C:\archive\Data\genres_original\blues\blues.00002.wav
Пропус

In [15]:
# Параметры фичей
SAMPLE_RATE = 22050
N_MFCC = 40
N_MELS = 128
HOP_LENGTH = 512
DURATION = 30  # seconds - GTZAN tracks 30s

def load_audio(path, sr=SAMPLE_RATE, duration=DURATION):
    try:
        y, _ = librosa.load(path, sr=sr, mono=True, duration=duration)
        return y
    except Exception as e:
        print(f"Ошибка загрузки файла {path}: {str(e)}")
        # Возвращаем нулевой сигнал той же длины, чтобы не прерывать обучение
        return np.zeros(sr * duration)

def extract_enhanced_features(y, sr=SAMPLE_RATE, n_mfcc=N_MFCC, n_mels=N_MELS, hop_length=HOP_LENGTH):
    # MFCC
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length)
    
    # Mel spectrogram
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_db = librosa.power_to_db(mel)
    
    # Chroma features
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop_length)
    
    # Spectral contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=hop_length)
    
    # Tonnetz features
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
    
    # Stack features: shape (channels, time)
    feat = np.vstack([mfcc, mel_db, chroma, spectral_contrast, tonnetz])
    return feat.astype(np.float32)
def augment_audio(y, sr=SAMPLE_RATE):
    augmented_versions = []
    
    # Оригинал
    augmented_versions.append(y)
    
    # Добавление шума
    noise = np.random.randn(len(y)) * 0.005
    augmented_versions.append(y + noise)
    
    # Изменение скорости
    speed_factor = np.random.uniform(0.9, 1.1)
    y_speed = librosa.effects.time_stretch(y, rate=speed_factor)
    if len(y_speed) < len(y):
        y_speed = np.pad(y_speed, (0, max(0, len(y) - len(y_speed))), mode='constant')
    else:
        y_speed = y_speed[:len(y)]
    augmented_versions.append(y_speed)
    
    # Изменение тона
    n_steps = np.random.randint(-2, 2)
    y_pitch = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)
    augmented_versions.append(y_pitch)
    
    return augmented_versions

# Модифицируем датасет для поддержки аугментации
class AugmentedMusicDataset(Dataset):
    def __init__(self, df, label2idx, transform=None, augment=False):
        self.df = df.reset_index(drop=True)
        self.label2idx = label2idx
        self.transform = transform
        self.augment = augment

    def __len__(self):
        return len(self.df) * 4 if self.augment else len(self.df)

    def __getitem__(self, idx):
        import gc
        gc.collect()
        
        if self.augment:
            orig_idx = idx // 4
            aug_type = idx % 4
        else:
            orig_idx = idx
            aug_type = 0
        
        row = self.df.iloc[orig_idx]
        path = row['path']
        label = self.label2idx[row['genre']]
        y = load_audio(path)
        
        # Применяем аугментацию
        if self.augment:
            augmented_versions = augment_audio(y)
            y = augmented_versions[aug_type]
        
        feat = extract_enhanced_features(y)
        
        # Ensure fixed length
        max_time = 800
        if feat.shape[1] < max_time:
            pad_width = max_time - feat.shape[1]
            feat = np.pad(feat, ((0, 0), (0, pad_width)), mode='constant')
        else:
            feat = feat[:, :max_time]
        
        # Normalize
        feat = (feat - feat.mean()) / (feat.std() + 1e-6)
        
        return torch.tensor(feat).unsqueeze(0), torch.tensor(label)

In [16]:
class MusicDataset(Dataset):
    def __init__(self, df, label2idx, transform=None):
        self.df = df.reset_index(drop=True)
        self.label2idx = label2idx
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        import gc
        gc.collect()  # Очистка памяти
        row = self.df.iloc[idx]
        path = row['path']
        label = self.label2idx[row['genre']]
        y = load_audio(path)
        feat = extract_enhanced_features(y)  # (channels, time)
        # Ensure fixed length in time dimension: pad or truncate
        max_time = 800
        if feat.shape[1] < max_time:
            pad_width = max_time - feat.shape[1]
            feat = np.pad(feat, ((0, 0), (0, pad_width)), mode='constant')
        else:
            feat = feat[:, :max_time]
        # Normalize per sample
        feat = (feat - feat.mean()) / (feat.std() + 1e-6)
        return torch.tensor(feat).unsqueeze(0), torch.tensor(label)

# Создаем label mapping
if 'df_bal' in globals():
    genres = sorted(df_bal['genre'].unique())
    label2idx = {g: i for i, g in enumerate(genres)}
    idx2label = {i: g for g, i in label2idx.items()}
    print('Genres:', genres)

# split
if 'df_bal' in globals():
    train_df, test_df = train_test_split(df_bal, test_size=0.2, stratify=df_bal['genre'], random_state=42)
    val_df, test_df = train_test_split(test_df, test_size=0.5, stratify=test_df['genre'], random_state=42)
    print(len(train_df), len(val_df), len(test_df))

    # Создаем аугментированные датасеты
    train_ds = AugmentedMusicDataset(train_df, label2idx, augment=True)
    val_ds = AugmentedMusicDataset(val_df, label2idx, augment=False)
    test_ds = AugmentedMusicDataset(test_df, label2idx, augment=False)

    # Создаем даталоадеры (убедитесь, что используете правильные параметры)
    # Уменьшите размер батча для экономии памяти
    BATCH_SIZE = 8  # вместо 16

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

Genres: []


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

проверка

In [21]:
# Проверка наличия FFmpeg
import subprocess
try:
    result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
    print("FFmpeg версия:", result.stdout.split('\n')[0] if result.stdout else "Не установлен")
except FileNotFoundError:
    print("FFmpeg не установлен или не добавлен в PATH")

FFmpeg версия: ffmpeg version 2025-09-10-git-c1dc2e2b7c-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers


In [20]:
import psutil
print(f"Memory usage: {psutil.virtual_memory().percent}%")

Memory usage: 69.4%


In [22]:
# ДОБАВЬТЕ ЭТУ ЯЧЕЙКУ ПОСЛЕ СОЗДАНИЯ DATALOADER'ОВ
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import librosa
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from torch.cuda.amp import autocast, GradScaler

# 1. Упрощенная модель с правильной инициализацией весов
class Simplified_CNN_RNN(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        
        # Упрощенная CNN часть
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Dropout2d(0.25),
            
            nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Dropout2d(0.25),
        )
        
        # Автоматический расчет размера
        with torch.no_grad():
            dummy_input = torch.randn(1, 1, 193, 1300)
            dummy_output = self.conv(dummy_input)
            self.gru_input_size = dummy_output.size(1) * dummy_output.size(2)
            self.gru_input_length = dummy_output.size(3)
        
        # Упрощенная RNN часть
        self.gru = nn.GRU(
            input_size=self.gru_input_size,
            hidden_size=128,
            batch_first=True,
            bidirectional=True,
            num_layers=1,
            dropout=0.3
        )
        
        # Упрощенные полносвязные слои
        self.fc = nn.Sequential(
            nn.Linear(128 * 2, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, n_classes)
        )
        
        # Инициализация весов
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.conv(x)
        batch_size, channels, height, width = x.size()
        x = x.permute(0, 3, 1, 2)
        x = x.contiguous().view(batch_size, width, channels * height)
        out, _ = self.gru(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# 2. Функции для проверки данных
def check_for_nan_inf(tensor, name):
    if torch.isnan(tensor).any():
        print(f"NaN detected in {name}")
        return True
    if torch.isinf(tensor).any():
        print(f"Inf detected in {name}")
        return True
    return False

def check_dataset(dataset, name="dataset"):
    print(f"Checking {name}...")
    for i in range(min(10, len(dataset))):
        x, y = dataset[i]
        if torch.isnan(x).any() or torch.isinf(x).any():
            print(f"NaN/Inf found in sample {i}")
        if x.max() > 1000 or x.min() < -1000:
            print(f"Extreme values in sample {i}: min={x.min()}, max={x.max()}")
    print(f"{name} check completed")

# 3. Модифицированные функции обучения и валидации
def train_epoch(model, loader, optimizer, criterion, scaler, device):
    model.train()
    losses = []
    ys, ys_pred = [], []
    
    for x, y in tqdm(loader, desc="Training"):
        x, y = x.to(device), y.to(device)
        
        # Проверка входных данных
        if check_for_nan_inf(x, "input data"):
            continue
            
        optimizer.zero_grad()
        
        with autocast():
            outputs = model(x)
            loss = criterion(outputs, y)
        
        # Проверка loss
        if torch.isnan(loss) or torch.isinf(loss):
            print("Skipping batch due to NaN/Inf loss")
            continue
            
        scaler.scale(loss).backward()
        
        # Градиентный клиппинг для предотвращения взрыва градиентов
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        scaler.step(optimizer)
        scaler.update()
        
        losses.append(loss.item())
        ys.extend(y.cpu().numpy())
        ys_pred.extend(outputs.argmax(dim=1).cpu().numpy())
    
    return np.mean(losses) if losses else float('nan'), ys, ys_pred

def validate(model, loader, criterion, device):
    model.eval()
    losses = []
    ys, ys_pred = [], []
    
    with torch.no_grad():
        for x, y in tqdm(loader, desc="Validation"):
            x, y = x.to(device), y.to(device)
            
            with autocast():
                outputs = model(x)
                loss = criterion(outputs, y)
            
            losses.append(loss.item())
            ys.extend(y.cpu().numpy())
            ys_pred.extend(outputs.argmax(dim=1).cpu().numpy())
    
    return np.mean(losses) if losses else float('nan'), ys, ys_pred

# Определение функции потерь 
criterion = nn.CrossEntropyLoss()

# 4. Основной цикл обучения с улучшениями
# Проверка данных перед началом обучения
print("Checking training data...")
check_dataset(train_ds, "training")
print("Checking validation data...")
check_dataset(val_ds, "validation")

# Инициализация модели с уменьшенной сложностью
model = Simplified_CNN_RNN(n_classes=len(label2idx)).to(DEVICE)

# Уменьшенный learning rate и вес decay
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3
)

# Mixed precision training
scaler = GradScaler()

# Ранняя остановка
early_stopping_patience = 5
best_val_loss = float('inf')
counter = 0

# Списки для отслеживания метрик
train_losses, val_losses = [], []
train_accs, val_accs = [], []

# Сохраняем начальную модель на случай прерывания обучения
if not os.path.exists('best_model.pth'):
    torch.save(model.state_dict(), 'best_model.pth')
    print("Создана начальная версия модели")

# Цикл обучения
n_epochs = 10
for epoch in range(n_epochs):
    print(f"\n--- Начало эпохи {epoch+1} ---")
    print_gpu_memory()
    
    # Training
    train_loss, y_train, y_train_pred = train_epoch(model, train_loader, optimizer, criterion, scaler, DEVICE)
    train_acc = accuracy_score(y_train, y_train_pred)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Validation
    val_loss, y_val, y_val_pred = validate(model, val_loader, criterion, DEVICE)
    val_acc = accuracy_score(y_val, y_val_pred)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    # Learning rate scheduling
    if not np.isnan(val_loss):
        scheduler.step(val_loss)
    
    print("После обучения:")
    print_gpu_memory()
    clear_gpu_memory()
    
    # Пропускаем эпоху, если loss NaN
    if np.isnan(train_loss) or np.isnan(val_loss):
        print(f"Epoch {epoch+1} skipped due to NaN loss")
        continue
    
    # Ранняя остановка
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"Новая лучшая модель сохранена с val_loss: {val_loss:.4f}")
    else:
        counter += 1
        if counter >= early_stopping_patience:
            print("Ранняя остановка!")
            break
    
    print(f"Epoch {epoch+1}/{n_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    
    # Визуализация матрицы ошибок каждые 2 эпохи
    if epoch % 2 == 0:
        cm = confusion_matrix(y_val, y_val_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=label2idx.keys(), 
                    yticklabels=label2idx.keys())
        plt.title(f'Confusion Matrix - Epoch {epoch+1}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()

# Загрузка лучшей модели
best_model_path = 'best_model.pth'
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path))
    print("Загружена лучшая модель из предыдущего обучения")
else:
    print("Предварительно обученная модель не найдена. Используется текущая модель.")

# Тестирование модели
test_loss, y_test, y_test_pred = validate(model, test_loader, criterion, DEVICE)
test_acc = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred, average='weighted')
print(f"\nTest Results: Accuracy: {test_acc:.4f} | F1: {test_f1:.4f}")

# Визуализация кривых обучения
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Validation Accuracy')
plt.title('Accuracy Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

Checking training data...
Checking training...
training check completed
Checking validation data...
Checking validation...
validation check completed


c:\Users\MIKHAIL\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(



--- Начало эпохи 1 ---
Использовано памяти GPU: 0.01 GB
Зарезервировано памяти GPU: 0.02 GB


C:\Users\MIKHAIL\AppData\Local\Temp\ipykernel_5836\1951357061.py:184: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Training:   0%|          | 0/800 [00:00<?, ?it/s]

C:\Users\MIKHAIL\AppData\Local\Temp\ipykernel_5836\1951357061.py:121: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


KeyboardInterrupt: 

Проверка работы модели

In [1]:
# Загрузите одну аудиозапись из тестового набора
test_sample, test_label = next(iter(test_loader))
test_sample = test_sample[0].unsqueeze(0).to(DEVICE)  # Берем первый sample из батча

# Предсказание
model.eval()
with torch.no_grad():
    output = model(test_sample)
    predicted = idx2label[output.argmax().item()]
    actual = idx2label[test_label[0].item()]

print(f"Predicted: {predicted}, Actual: {actual}")

NameError: name 'test_loader' is not defined

Сохраните метки в файл labels.json

In [26]:
# Создайте файл labels.json с правильными метками
import json

labels = {
    "label2idx": {
        "blues": 0,
        "classical": 1,
        "country": 2,
        "disco": 3,
        "hiphop": 4,
        "jazz": 5,
        "metal": 6,
        "pop": 7,
        "reggae": 8,
        "rock": 9
    },
    "idx2label": {
        "0": "blues",
        "1": "classical",
        "2": "country",
        "3": "disco",
        "4": "hiphop",
        "5": "jazz",
        "6": "metal",
        "7": "pop",
        "8": "reggae",
        "9": "rock"
    }
}

with open('labels.json', 'w') as f:
    json.dump(labels, f)

In [27]:
import json

# Сохраните метки
with open('labels.json', 'w') as f:
    json.dump({
        'label2idx': label2idx,
        'idx2label': {k: v for k, v in idx2label.items()}
    }, f)

In [1]:
# Убедитесь, что установлена правильная библиотека
!pip uninstall telegram
!pip install python-telegram-bot


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import os, urllib.request, json

# 1) ВСТАВЬТЕ НОВЫЙ ТОКЕН
os.environ["TELEGRAM_TOKEN"] = "8137581059:AAFOSgGg1RE-Kq0coZceSYncj_L89DMdYK0"  # <-- замените на ваш актуальный токен

print("TELEGRAM_TOKEN set:", bool(os.environ.get("TELEGRAM_TOKEN")))

# 2) Сброс webhook и очистка очереди апдейтов
with urllib.request.urlopen(f"https://api.telegram.org/bot{os.environ['TELEGRAM_TOKEN']}/deleteWebhook?drop_pending_updates=true") as r:
    print("deleteWebhook:", json.load(r))

# 3) Быстрая проверка токена
with urllib.request.urlopen(f"https://api.telegram.org/bot{os.environ['TELEGRAM_TOKEN']}/getMe") as r:
    print("getMe:", json.load(r))

TELEGRAM_TOKEN set: True
deleteWebhook: {'ok': True, 'result': True, 'description': 'Webhook is already deleted'}
getMe: {'ok': True, 'result': {'id': 8137581059, 'is_bot': True, 'first_name': 'music_genre_classifier_bot', 'username': 'mgc_2025_bot', 'can_join_groups': True, 'can_read_all_group_messages': True, 'supports_inline_queries': False, 'can_connect_to_business': False, 'has_main_web_app': False}}


In [9]:
# One-click: API (если нужно) + БОТ с логами (видно причину, если упадёт)
import os, sys, time, socket, subprocess, urllib.request, json
from pathlib import Path

# Где лежат api.py и bot.py
candidates = [Path.cwd(), Path(r"C:\MT"), Path.home() / "music-bot"]
PROJECT_DIR = next((p for p in candidates if (p/"api.py").exists() and (p/"bot.py").exists()), None)
if not PROJECT_DIR:
    raise RuntimeError("Не нашёл api.py и bot.py. Положите их в текущую папку, C:\\MT или %USERPROFILE%\\music-bot")

# Проверим/поставим python-telegram-bot нужной версии (21.x)
try:
    import telegram
    print("python-telegram-bot:", getattr(telegram, "__version__", "unknown"))
except Exception:
    print("Устанавливаю python-telegram-bot...")
    import sys as _sys, subprocess as _sub
    _sub.check_call([_sys.executable, "-m", "pip", "install", "-U", "python-telegram-bot==21.6"])
    import telegram
    print("python-telegram-bot:", getattr(telegram, "__version__", "unknown"))

# Готовим окружение
env = os.environ.copy()
if not env.get("TELEGRAM_TOKEN"):
    raise RuntimeError("TELEGRAM_TOKEN не установлен в этом ядре. Запустите предыдущую ячейку с токеном.")

# Проверим, жив ли API; если нет — поднимем на 8000
PORT = int(env.get("PORT", "8000"))
base = f"http://127.0.0.1:{PORT}"

def api_alive():
    try:
        with urllib.request.urlopen(base + "/") as r:
            return r.status == 200
    except Exception:
        return False

api_proc = None
if not api_alive():
    print("API не найден — запускаю на порту", PORT)
    api_proc = subprocess.Popen([sys.executable, "api.py"], cwd=str(PROJECT_DIR), env=env)
    print("API PID:", api_proc.pid)
    for _ in range(60):
        if api_alive():
            print("API is up:", base)
            break
        time.sleep(0.5)
    else:
        if api_proc:
            api_proc.kill()
        raise RuntimeError("API не поднялся. Посмотрите логи api.py вручную: !python api.py")

# Установим URL для бота
env["API_URL"] = f"{base}/predict"
print("API_URL для бота:", env["API_URL"])

# Запускаем бота с ПОЛНЫМИ логами прямо здесь
print("Старт бота... (для остановки прервите выполнение ячейки)")
bot = subprocess.Popen([sys.executable, "bot.py"], cwd=str(PROJECT_DIR), env=env)
print("Bot PID:", bot.pid)

# Подождём немного и посмотрим, жив ли процесс
time.sleep(2)
ret = bot.poll()
if ret is not None:
    print("Бот завершился мгновенно, код:", ret)
    # Считаем stdout/stderr для диагностики
    out = subprocess.run([sys.executable, "bot.py"], cwd=str(PROJECT_DIR), env=env, capture_output=True, text=True)
    print("\n=== bot.py STDOUT ===\n", out.stdout)
    print("\n=== bot.py STDERR ===\n", out.stderr)
    print("\nКод возврата:", out.returncode)
    raise SystemExit
else:
    print("Бот запущен. Напишите ему /start и отправьте аудио.")

python-telegram-bot: 22.4
API_URL для бота: http://127.0.0.1:8000/predict
Старт бота... (для остановки прервите выполнение ячейки)
Bot PID: 24480
Бот запущен. Напишите ему /start и отправьте аудио.


In [15]:
try:
    bot.terminate()
    print("Bot terminated.")
except Exception as e:
    print("Bot stop:", e)

Bot terminated.
